# Concurrent Data Structure Performance Analysis

This notebook provides interactive analysis of Tier 3 core algorithm performance:
- Skip List (lock-free vs locked)
- BST (lock-free vs locked)
- Stack (Treiber with elimination)

Use this to evaluate which data structure best fits your workload.

In [ ]:
import time
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from dataclasses import dataclass
from typing import Dict, List, Callable
import matplotlib.pyplot as plt
import numpy as np

# Try to import real library, fall back to simulation
try:
    from concurrent_collections import SkipListMap, TreeMap, LockFreeStack
    from concurrent_collections.profilers import SkipListProfiler, BSTProfiler, StackProfiler
    SIMULATION_MODE = False
    print("Using concurrent_collections library")
except ImportError:
    SIMULATION_MODE = True
    print("Library not installed - running in simulation mode")

In [ ]:
# Simulation classes for demonstration
if SIMULATION_MODE:
    @dataclass
    class SimulatedStats:
        total_inserts: int = 0
        total_deletes: int = 0
        total_searches: int = 0
        cas_failure_rate: float = 0.0
        insert_latency_p50: float = 0.0
        insert_latency_p99: float = 0.0
        search_latency_p50: float = 0.0
        search_latency_p99: float = 0.0
    
    class SimulatedMap(dict):
        def __init__(self, name="SimulatedMap"):
            super().__init__()
            self.name = name
            self._lock = threading.Lock()
        
        def put(self, key, value):
            with self._lock:
                self[key] = value
        
        def pop(self, key, default=None):
            with self._lock:
                return super().pop(key, default)
    
    class SimulatedStack:
        def __init__(self):
            self._items = []
            self._lock = threading.Lock()
        
        def push(self, item):
            with self._lock:
                self._items.append(item)
        
        def pop(self):
            with self._lock:
                return self._items.pop() if self._items else None
    
    SkipListMap = lambda: SimulatedMap("SkipListMap")
    TreeMap = lambda: SimulatedMap("TreeMap")
    LockFreeStack = SimulatedStack

## Benchmark Framework

In [ ]:
@dataclass
class BenchmarkResult:
    name: str
    threads: int
    ops_per_sec: float
    latency_p50_us: float
    latency_p99_us: float
    total_ops: int
    duration_sec: float

def benchmark_map(map_factory: Callable, name: str, threads: int,
                  ops_per_thread: int = 10000,
                  read_ratio: float = 0.8) -> BenchmarkResult:
    """Benchmark a map implementation."""
    m = map_factory()
    latencies = []
    total_ops = [0]
    
    # Pre-populate
    for i in range(1000):
        m.put(i, i)
    
    def worker():
        local_latencies = []
        for _ in range(ops_per_thread):
            key = random.randint(0, 2000)
            start = time.perf_counter_ns()
            
            if random.random() < read_ratio:
                _ = m.get(key)
            elif random.random() < 0.5:
                m.put(key, key)
            else:
                m.pop(key, None)
            
            elapsed = (time.perf_counter_ns() - start) / 1000
            local_latencies.append(elapsed)
        
        with threading.Lock():
            latencies.extend(local_latencies)
            total_ops[0] += ops_per_thread
    
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=threads) as executor:
        futures = [executor.submit(worker) for _ in range(threads)]
        for f in futures:
            f.result()
    duration = time.time() - start_time
    
    sorted_lat = sorted(latencies)
    n = len(sorted_lat)
    
    return BenchmarkResult(
        name=name,
        threads=threads,
        ops_per_sec=total_ops[0] / duration,
        latency_p50_us=sorted_lat[int(n * 0.50)] if n > 0 else 0,
        latency_p99_us=sorted_lat[min(int(n * 0.99), n-1)] if n > 0 else 0,
        total_ops=total_ops[0],
        duration_sec=duration
    )

## Skip List vs BST Comparison

In [ ]:
# Compare Skip List and BST across thread counts
thread_counts = [1, 2, 4, 8]
skiplist_results = []
tree_results = []

print("Running benchmarks...")
for t in thread_counts:
    print(f"  {t} threads...")
    skiplist_results.append(benchmark_map(SkipListMap, "SkipListMap", t))
    tree_results.append(benchmark_map(TreeMap, "TreeMap", t))

print("Done!")

In [ ]:
# Throughput comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Throughput
ax1 = axes[0]
x = np.arange(len(thread_counts))
width = 0.35

skiplist_throughput = [r.ops_per_sec / 1000 for r in skiplist_results]
tree_throughput = [r.ops_per_sec / 1000 for r in tree_results]

bars1 = ax1.bar(x - width/2, skiplist_throughput, width, label='SkipListMap', color='steelblue')
bars2 = ax1.bar(x + width/2, tree_throughput, width, label='TreeMap', color='coral')

ax1.set_xlabel('Thread Count')
ax1.set_ylabel('Throughput (K ops/sec)')
ax1.set_title('Throughput Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(thread_counts)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Latency
ax2 = axes[1]
skiplist_p99 = [r.latency_p99_us for r in skiplist_results]
tree_p99 = [r.latency_p99_us for r in tree_results]

ax2.plot(thread_counts, skiplist_p99, 'o-', label='SkipListMap P99', color='steelblue')
ax2.plot(thread_counts, tree_p99, 's-', label='TreeMap P99', color='coral')

ax2.set_xlabel('Thread Count')
ax2.set_ylabel('Latency (μs)')
ax2.set_title('P99 Latency Comparison')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Workload Analysis

In [ ]:
# Test different read/write ratios
read_ratios = [0.5, 0.8, 0.95, 0.99]
skiplist_by_ratio = []
tree_by_ratio = []

print("Testing workload ratios (4 threads)...")
for ratio in read_ratios:
    print(f"  {int(ratio*100)}% reads...")
    skiplist_by_ratio.append(benchmark_map(SkipListMap, "SkipList", 4, read_ratio=ratio))
    tree_by_ratio.append(benchmark_map(TreeMap, "Tree", 4, read_ratio=ratio))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

x = [f"{int(r*100)}%" for r in read_ratios]
skiplist_tp = [r.ops_per_sec / 1000 for r in skiplist_by_ratio]
tree_tp = [r.ops_per_sec / 1000 for r in tree_by_ratio]

x_pos = np.arange(len(read_ratios))
ax.bar(x_pos - 0.2, skiplist_tp, 0.4, label='SkipListMap', color='steelblue')
ax.bar(x_pos + 0.2, tree_tp, 0.4, label='TreeMap', color='coral')

ax.set_xlabel('Read Ratio')
ax.set_ylabel('Throughput (K ops/sec)')
ax.set_title('Performance by Workload Type')
ax.set_xticks(x_pos)
ax.set_xticklabels(x)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Stack Performance with Elimination

In [ ]:
def benchmark_stack(stack_factory: Callable, name: str, threads: int,
                    ops_per_thread: int = 10000) -> BenchmarkResult:
    """Benchmark a stack implementation."""
    stack = stack_factory()
    latencies = []
    total_ops = [0]
    
    def worker():
        local_latencies = []
        for i in range(ops_per_thread):
            start = time.perf_counter_ns()
            
            if i % 2 == 0:
                stack.push(i)
            else:
                stack.pop()
            
            elapsed = (time.perf_counter_ns() - start) / 1000
            local_latencies.append(elapsed)
        
        with threading.Lock():
            latencies.extend(local_latencies)
            total_ops[0] += ops_per_thread
    
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=threads) as executor:
        futures = [executor.submit(worker) for _ in range(threads)]
        for f in futures:
            f.result()
    duration = time.time() - start_time
    
    sorted_lat = sorted(latencies)
    n = len(sorted_lat)
    
    return BenchmarkResult(
        name=name,
        threads=threads,
        ops_per_sec=total_ops[0] / duration,
        latency_p50_us=sorted_lat[int(n * 0.50)] if n > 0 else 0,
        latency_p99_us=sorted_lat[min(int(n * 0.99), n-1)] if n > 0 else 0,
        total_ops=total_ops[0],
        duration_sec=duration
    )

# Benchmark stack
stack_results = []
print("Benchmarking stack...")
for t in thread_counts:
    print(f"  {t} threads...")
    stack_results.append(benchmark_stack(LockFreeStack, "LockFreeStack", t))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

stack_tp = [r.ops_per_sec / 1000 for r in stack_results]

ax.bar(thread_counts, stack_tp, color='forestgreen', alpha=0.8)
ax.set_xlabel('Thread Count')
ax.set_ylabel('Throughput (K ops/sec)')
ax.set_title('Lock-Free Stack Performance')
ax.grid(axis='y', alpha=0.3)

# Annotate with P99 latency
for i, (t, r) in enumerate(zip(thread_counts, stack_results)):
    ax.annotate(f'P99: {r.latency_p99_us:.1f}μs',
                xy=(t, stack_tp[i]), ha='center', va='bottom',
                fontsize=9, color='darkgreen')

plt.tight_layout()
plt.show()

## Recommendations

In [ ]:
def generate_recommendations(skiplist_results, tree_results, stack_results):
    """Generate data structure recommendations based on benchmark results."""
    recommendations = []
    
    # Compare at highest thread count
    sl = skiplist_results[-1]
    tr = tree_results[-1]
    st = stack_results[-1]
    
    # Ordered map recommendation
    if sl.ops_per_sec > tr.ops_per_sec * 1.1:
        recommendations.append({
            'category': 'Ordered Map',
            'recommendation': 'SkipListMap',
            'reason': f'{sl.ops_per_sec/tr.ops_per_sec:.1f}x faster at {sl.threads} threads'
        })
    elif tr.ops_per_sec > sl.ops_per_sec * 1.1:
        recommendations.append({
            'category': 'Ordered Map',
            'recommendation': 'TreeMap',
            'reason': f'{tr.ops_per_sec/sl.ops_per_sec:.1f}x faster at {tr.threads} threads'
        })
    else:
        recommendations.append({
            'category': 'Ordered Map',
            'recommendation': 'Either (similar performance)',
            'reason': 'Choose based on range query needs (SkipList better for ranges)'
        })
    
    # Latency analysis
    if sl.latency_p99_us < tr.latency_p99_us:
        recommendations.append({
            'category': 'Latency-Sensitive',
            'recommendation': 'SkipListMap',
            'reason': f'P99 latency {sl.latency_p99_us:.1f}μs vs {tr.latency_p99_us:.1f}μs'
        })
    
    return recommendations

recommendations = generate_recommendations(skiplist_results, tree_results, stack_results)

print("\n" + "="*60)
print("RECOMMENDATIONS")
print("="*60)
for rec in recommendations:
    print(f"\n{rec['category']}:")
    print(f"  Recommended: {rec['recommendation']}")
    print(f"  Reason: {rec['reason']}")

## Summary Table

In [ ]:
print("\nBenchmark Summary")
print("="*80)
print(f"{'Data Structure':<20} {'Threads':>8} {'Throughput':>15} {'P50 (μs)':>12} {'P99 (μs)':>12}")
print("-"*80)

for r in skiplist_results:
    print(f"{r.name:<20} {r.threads:>8} {r.ops_per_sec/1000:>12.1f} K/s {r.latency_p50_us:>12.2f} {r.latency_p99_us:>12.2f}")

print()
for r in tree_results:
    print(f"{r.name:<20} {r.threads:>8} {r.ops_per_sec/1000:>12.1f} K/s {r.latency_p50_us:>12.2f} {r.latency_p99_us:>12.2f}")

print()
for r in stack_results:
    print(f"{r.name:<20} {r.threads:>8} {r.ops_per_sec/1000:>12.1f} K/s {r.latency_p50_us:>12.2f} {r.latency_p99_us:>12.2f}")